# Testing of imgaging.py

In [1]:
import numpy as np
import sys
import os
# we are in alabtools/test/imaging_testing
# we have to import functions in alabtools/
sys.path.append(os.path.abspath('../..'))
from alabtools.utils import Genome, Index
from alabtools.imaging import CtFile

The standardized data by 4DN for Chromatin Tracing is FOF-CT (Fish Omics Format - Chromatin Tracing).

I have downloaded a few datasets from different labs to test the code.

I have also created two cvs test files, with just a few lines, so that we can test the output.

First, I am going to read the data from one test file.

In [2]:
if os.path.exists('ct_test1.ct'):
    os.system('rm ct_test1.ct')  # remove the file if it exists
# Reading the test csv file 1
ct = CtFile('ct_test1.ct', 'w')  # initialize the ct file
ct.set_from_fofct('./data/csv_test_1.csv')  # read the FOFCT file

Assembly hg38 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


A CT file is very similar to an HSS one. It is an HDF5 file, and has a genome and an index objects as data.

There is an important difference though. HDF5 files support only homogeneous arrays (i.e. hyperrectangules) as dataset. For Chromatin Tracing, the data is inherently heterogeneous: for example, the number of spots can change by copy/domain/cell. Here this issue is solved by max-padding: the dimensions are increased to match the maximum values, and the rest is set to NaN. This is applied to both coordinates and number of spots, and consequently there are two attributes (nspot_max and ncopy_max) that are used for max-padding only.

CT attributes:

    - ncell : number of cells
    - ndomain : number of domains (in Index object)
    - nspot_tot : total number of spots measured
    - ntrace_tot : total number of chromatin traces identified
    - nspot_max : maximum number of spots per copy/domain/cell
    - ncopy_max : maximum number of copies per domain/cell

CT datasets:

    - cell_labels : np.array(ncell, dtype='str'), contains cellIDs
    - coordinates : np.array(ncell, ndomain, ncopy_max, nspot_max, 3)
    - nspot: np.array(ncell, ndomain, ncopy_max), number of spots per copy/domain/cell
    - ncopy: np.array(ncell, ndomain), number of copies per domain/cell

In [3]:
print('ct.ncell = {}'.format(ct.ncell))
print('ct.ndomain = {}'.format(ct.ndomain))
print('ct.nspot_tot = {}'.format(ct.nspot_tot))
print('ct.ntrace_tot = {}'.format(ct.ntrace_tot))
print('ct.nspot_max = {}'.format(ct.nspot_max))
print('ct.ncopy_max = {}'.format(ct.ncopy_max))
print('ct.cell_labels.shape = {}'.format(ct.cell_labels.shape))
print('ct.coordinates.shape = {}'.format(ct.coordinates.shape)) 
print('ct.nspot.shape = {}'.format(ct.nspot.shape))
print('ct.ncopy.shape = {}'.format(ct.ncopy.shape))

ct.ncell = 4
ct.ndomain = 4
ct.nspot_tot = 5
ct.ntrace_tot = 4
ct.nspot_max = 1
ct.ncopy_max = 1
ct.cell_labels.shape = (4,)
ct.coordinates.shape = (4, 4, 1, 1, 3)
ct.nspot.shape = (4, 4, 1)
ct.ncopy.shape = (4, 4)


In [4]:
# Test data has been imported correctly
print(ct.genome.assembly == 'hg38')
print(np.all(ct.index.chromstr == np.array(['chr1', 'chr1', 'chr4', 'chr7'])))
print(np.all(ct.index.start == np.array([0, 10000, 10000, 5000])))
print(np.all(ct.index.end == np.array([10000, 22000, 20500, 15000])))
print(ct.coordinates)

True
True
True
True
[[[[[ 1.   2.   3. ]]]


  [[[ 8.   9.   1.5]]]


  [[[ nan  nan  nan]]]


  [[[ nan  nan  nan]]]]



 [[[[ nan  nan  nan]]]


  [[[ 3.   7.   5. ]]]


  [[[ nan  nan  nan]]]


  [[[ nan  nan  nan]]]]



 [[[[ nan  nan  nan]]]


  [[[ nan  nan  nan]]]


  [[[ 6.   3.4  9. ]]]


  [[[ nan  nan  nan]]]]



 [[[[ nan  nan  nan]]]


  [[[ nan  nan  nan]]]


  [[[ nan  nan  nan]]]


  [[[11.   3.   8. ]]]]]


The header of csv_test_1 was complete.

Some datasets don't have a complete header, only having the column names. In this case it's necessary to include the assembly manually.

I have coded these cases as well, and created a csv test 2 to check that everything works correctly.

In [5]:
if os.path.exists('ct_test2.ct'):
    os.system('rm ct_test2.ct')  # remove the file if it exists
# Reading the test csv file 2
ct2 = CtFile('ct_test2.ct', 'w')
ct2.set_from_fofct('./data/csv_test_2.csv', 'hg38')

Assembly not found in FOF-CT file. Using the one provided.
Assembly hg38 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


In [6]:
# Test data has been imported correctly
print(ct2.genome.assembly == 'hg38')
print(np.all(ct2.index.chromstr == np.array(['chr1', 'chr2'])))
print(np.all(ct2.index.start == np.array([0, 5000])))
print(np.all(ct2.index.end == np.array([5000, 11200])))
print(ct2.coordinates)

True
True
True
True
[[[[[1.1 2.  3. ]]]


  [[[nan nan nan]]]]



 [[[[nan nan nan]]]


  [[[3.5 7.  9.1]]]]



 [[[[nan nan nan]]]


  [[[5.4 6.  8.9]]]]]


There is also a merging feature. This should be used when we combine independent data of the same experiment, e.g. biological replicates.

The merging requires that the Genome and the Index objects are the same. Additionaly, it requires that there is no overlab between the cell_labels: if there is, users should input tags to distinguish between the replicates.

In [7]:
# Try merging ct and ct2: fails because genome/index are different
if os.path.exists('ct_test3.ct'):
    os.system('rm ct_test3.ct')  # remove the file if it exists
ct3 = ct.merge(ct2, 'ct_test3.ct')  # merge the two ct files

ValueError: Cannot merge CtFile instances with different genomes.

In [8]:
# Merge ct with itself with tags
if os.path.exists('ct_test3.ct'):
    os.system('rm ct_test3.ct')  # remove the file if it exists
ct3 = ct.merge(ct, 'ct_test3.ct', '_rep1', '_rep2')  # merge the two ct files

In [9]:
# Test meging was correct
print(ct3.genome.assembly == 'hg38')
print(np.all(ct3.index.chromstr == np.array(['chr1', 'chr1', 'chr4', 'chr7'])))
print(np.all(ct3.index.start == np.array([0, 10000, 10000, 5000])))
print(np.all(ct3.index.end == np.array([10000, 22000, 20500, 15000])))
print(ct3.coordinates)

True
True
True
True
[[[[[ 1.         2.         3.       ]]]


  [[[ 8.         9.         1.5      ]]]


  [[[       nan        nan        nan]]]


  [[[       nan        nan        nan]]]]



 [[[[       nan        nan        nan]]]


  [[[ 3.         7.         5.       ]]]


  [[[       nan        nan        nan]]]


  [[[       nan        nan        nan]]]]



 [[[[       nan        nan        nan]]]


  [[[       nan        nan        nan]]]


  [[[ 6.         3.4000001  9.       ]]]


  [[[       nan        nan        nan]]]]



 [[[[       nan        nan        nan]]]


  [[[       nan        nan        nan]]]


  [[[       nan        nan        nan]]]


  [[[11.         3.         8.       ]]]]



 [[[[ 1.         2.         3.       ]]]


  [[[ 8.         9.         1.5      ]]]


  [[[       nan        nan        nan]]]


  [[[       nan        nan        nan]]]]



 [[[[       nan        nan        nan]]]


  [[[ 3.         7.         5.       ]]]


  [[[       nan        n

In [10]:
ct.close()
ct2.close()
ct3.close()

Now we can load the actual file and check that everything works.

In [11]:
if os.path.exists('ct_takei_rep1.ct'):
    os.system('rm ct_takei_rep1.ct')  # remove the file if it exists
ct = CtFile('ct_takei_rep1.ct', 'w')
ct.set_from_fofct('./data/takei_mesc_DNA-tracing_1Mb_rep1.csv')

Assembly mm10 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


In [12]:
if os.path.exists('ct_takei_rep2.ct'):
    os.system('rm ct_takei_rep2.ct')  # remove the file if it exists
ct2 = CtFile('ct_takei_rep2.ct', 'w')
ct2.set_from_fofct('./data/takei_mesc_DNA-tracing_1Mb_rep2.csv')

Assembly mm10 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


In [16]:
ct3 = ct.merge(ct2, 'ct_takei_comb.ct', '_rep1', '_rep2')  # merge the two ct files

In [17]:
print(ct3.ncell)
print(ct3.ndomain)
print(ct3.nspot_tot)
print(ct3.ntrace_tot)
print(ct3.nspot_max)
print(ct3.ncopy_max)
print(ct3.cell_labels.shape)
print(ct3.coordinates.shape)
print(ct3.nspot.shape)
print(ct3.ncopy.shape)


446
2460
1795228
8919
15
1
(446,)
(446, 2460, 1, 15, 3)
(446, 2460, 1)
(446, 2460)


In [19]:
# Check that the Index object is correctly created from the FOFCT file

i = Index('./data/domains_takei.bed', ct3.genome)

print(i == ct3.index)

ct.close()
ct2.close()
ct3.close()

True


Try other datasets. I just want to check that they are imported correctly, or that the appropriate error message is shown.

In [20]:
# Data from Takei (25kb)

if os.path.exists('ct_takei25_rep1.ct'):
    os.system('rm ct_takei25_rep1.ct')  # remove the file if it exists
ct = CtFile('ct_takei25_rep1.ct', 'w')
ct.set_from_fofct('./data/takei_mesc_DNA-tracing_25kb_rep1.csv')

print(ct.coordinates.shape)

Assembly mm10 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


(201, 1200, 4, 6, 3)


The shape of the coordinates array is (201, 1200, 4, 6, 3).

I was surprised to see that ncopy_max = 4, because Takei assumes there are only 2 copies.

We can check the cellID and the chromosome of the domains with 4 copies (see below).

These domains are in cell 0_44 chr11 and 4_11 chr18.
I checked on the cvs file that these spots are indeed traced in 4 copies (see lines 62780 and 276439).
I also checked that some spot have a -1 trace (see line 1527, traceID=0_2_1_-1).
This is likely a mistake, but I don't think we can code to account for this.

In [21]:
print(np.where(ct.ncopy == 4))
for cell_idx, dom_idx in zip(*np.where(ct.ncopy == 4)):
    print(cell_idx, dom_idx, ct.cell_labels[cell_idx], ct.index.chromstr[dom_idx])

(array([ 38,  38,  38,  38,  38,  75,  75, 172, 172, 172, 172, 172, 172]), array([ 633,  636,  638,  644,  645,  632, 1037, 1032, 1033, 1048, 1068,
       1072, 1079]))
38 633 0_44 chr11
38 636 0_44 chr11
38 638 0_44 chr11
38 644 0_44 chr11
38 645 0_44 chr11
75 632 1_36 chr11
75 1037 1_36 chr18
172 1032 4_11 chr18
172 1033 4_11 chr18
172 1048 4_11 chr18
172 1068 4_11 chr18
172 1072 4_11 chr18
172 1079 4_11 chr18


In [22]:
ct.close()

In [23]:
# Data from Bing Ren's lab

if os.path.exists('ct_bingren.ct'):
    os.system('rm ct_bingren.ct')  # remove the file if it exists
ct = CtFile('ct_bingren.ct', 'w')
ct.set_from_fofct('./data/bingren_4DNFIKPGMZJ8.csv')

print(ct.coordinates.shape)
ct.close()

Assembly grcm38 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


(752, 42, 2, 1, 3)


In [24]:
# Data from Boettiger's lab

# This dataset is for Drosophila melanogaster (dm6), which is not in alabtools/genomes. So we raise an error.

if os.path.exists('ct_boettiger.ct'):
    os.system('rm ct_boettiger.ct')  # remove the file if it exists
ct = CtFile('ct_boettiger.ct', 'w')
ct.set_from_fofct('./data/boettiger_4DNFI5WLD5KM.csv')

print(ct.coordinates.shape)
ct.close()

ValueError: Assembly dm6 not found in alabtools/genomes. Need to include it.

In [25]:
# Another dataset from Boettiger's lab, this time for human (hg38)

# The dataset doesn't have a proper header, but it's still imported if we specify the genome

# Furthermore, the dataset doesn't have a cellID column: that's because only one chromosome per cell
#   is imaged, so the cellID is redundant.

if os.path.exists('ct_boettiger.ct'):
    os.system('rm ct_boettiger.ct')  # remove the file if it exists
ct = CtFile('ct_boettiger.ct', 'w')
ct.set_from_fofct('./data/boettiger_4DNFIHZRN68I.csv', 'hg38')

print(ct.coordinates.shape)
ct.close()

Assembly not found in FOF-CT file. Using the one provided.
Assembly hg38 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


(9526, 83, 1, 1, 3)


In [26]:
# Dataset from Wang's lab

# Here as well there isn't a cellID column

if os.path.exists('ct_wang.ct'):
    os.system('rm ct_wang.ct')  # remove the file if it exists
ct = CtFile('ct_wang.ct', 'w')
ct.set_from_fofct('./data/wang_4DNFI7PBQK6G.csv')

print(ct.coordinates.shape)

Assembly grch38 found in alabtools/genomes. Using this.


chroms or lengths not given, reading from genomes info file.


(1038, 28, 1, 1, 3)
